running code invoves using scraper API, which requires api keys so may not be reproducable

# Summary
This notebook scrapes old listing data. It involves:
1. Importing necessary libraries.
2. Loading JSON and CSV data.
3. Fetching and parsing HTML content.
4. Generating URLs based on postcodes.
5. Defining functions to extract property information.
6. Scraping property data and storing it.

In [35]:
import re, sys, os
from json import dump
from tqdm import tqdm
import time
from collections import defaultdict
import urllib.request
import requests

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request

sys.path.append('../')
from scripts.oldlistings import get_oldlistings_page
import pandas as pd
import json

# https://github.com/Elkfox/Australian-Postcode-Data/blob/master/au_postcodes.csv source of postcodes

In [36]:
with open('../data/landing/example_postcode.jsonl', 'r') as f:
    data = json.load(f)
    


In [ ]:
data

In [ ]:
# get the list of postcodes defined by oldlistings
# template is https://www.oldlistings.com.au/site-map?page=17&state=VIC


url = "https://www.oldlistings.com.au/site-map?page=17&state=VIC"
response = requests.get(url)

Soup = BeautifulSoup(response.content, 'html.parser')
Soup

In [13]:
postcode = pd.read_csv("../data/landing/au_postcodes.csv")

In [ ]:

postcode = postcode[postcode["state_code"] == "VIC"]
# restrict the postcode between 3000 and 4000
postcode = postcode[postcode["postcode"] >= 3000]
postcode = postcode[postcode["postcode"] < 4000]
len(postcode)

In [31]:
# create a new dataframe for urls
url_list = []
df = pd.DataFrame(columns=["url"])

# format is https://www.oldlistings.com.au/real-estate/VIC/place_name/postcode/buy/
# if place name has space, rplace it with + sign

for i in range(len(postcode)):
    place_name = postcode.iloc[i]["place_name"]
    place_name = place_name.replace(" ", "+")
    postcode_number = postcode.iloc[i]["postcode"]
    url = f"https://www.oldlistings.com.au/real-estate/VIC/{place_name}/{postcode_number}/buy/"
    url_list.append(url)

df["url"] = url_list

# remove the header url

df.to_csv("../data/curated/oldlistings_urls.csv", index=False)


In [6]:
# this is for testing using downloaded html
# please use normal url for production

# read html with bs4
path = "../data/landing/South Melbourne Rent Real Estate Old Listings.html"

url = "https://www.oldlistings.com.au/real-estate/VIC/South+Melbourne/3205/buy/"
response = requests.get(url)

"""with open(path, 'r') as f:
    webpage = f.read()

soup = BeautifulSoup(webpage)"""
Soup = BeautifulSoup(response.content, 'html.parser')

# function to get info from a property

In [3]:
def get_info(p):
    # try to get data in json format 
    info_dict = defaultdict(None)
    info_dict['lat'] = p.get('data-lat')
    info_dict['lng'] = p.get('data-lng')
    info_dict['rented_prices'] = []

    p = p.find('section', {'class':"grid-100 grid-parent"}) # replace p with its only (useful)child

    title = p.find('section', {'class':"grid-65 tablet-grid-65 clearfix"})
    ad = p.find('section', {'class':"grid-35 tablet-grid-35 price"})
    list_of_history = p.find('section', {'class':"grid-100 historical-price"})


    # title
    info_dict['address'] = title.find('h2', {'class': 'address'}).text
    # other metadata: bed, bath, car etc
    # this do not garentee any structure of the data
    info_dict['meta_data'] = []
    for meta_data in title.find_all('p', {'class': re.compile("property-meta")}):
        obj = {}
        obj['label'] = meta_data.get('class')[1]
        obj['description'] = meta_data.find('span').text.split(' ')[0]
        obj['quantity'] = meta_data.contents[1].strip()
        info_dict['meta_data'].append(obj)


    # ad : we dont use ad

    # list of history
    for line in list_of_history.find('ul').find_all('li'):
        record = {}
        record['date'] = line.find('span').text
        record['price'] = line.contents[1]
        info_dict['rented_prices'].append(record)

    return info_dict

In [4]:
# soup.find_all('div', {'class':"content-col"})
def scrape_page(soup):
    properties = soup.find('div', {'class':"content-col"}).findChildren("div" , recursive=False)
    list_of_properties = []
    for property in properties:
        list_of_properties.append(get_info(property))
        # get_info(property)
    return list_of_properties

In [ ]:
print(Soup)

In [ ]:
properties = scrape_page(Soup)

In [ ]:
print(properties)